# Weekly project

Today you are going to implement the last parts of the algorithm you started on monday. For reference you can see it below.

![title](algorithm_3.png)

It is a good idea to follow and track the steps in the algorithm in the below implementation. Only take one step at a time.

Once you have the algorithm up and running you can try with a larger dataset to see if your algorithm is able to maintain good accurracy over a longer distance. The larger dataset can be found here:
[Left images](https://dtudk-my.sharepoint.com/:u:/g/personal/evanb_dtu_dk/EQu8kmGBDDROtGJ7IkZB2tQBJrxmgY9t8LVM_JuEi83TYw)
[Right images](https://dtudk-my.sharepoint.com/:u:/g/personal/evanb_dtu_dk/EcKI_zrXTvpMulizidCZm4oBLJcQ_LTV9Zs6oQFF74JTRQ)

In [1]:
import numpy as np
import cv2 as cv2
from numpy.linalg import inv, pinv
import matplotlib.pyplot as plt
import time as t
from helpers import *

def extract_keypoints_surf(img1, img2, K, baseline):
    """
    use surf to detect keypoint features
    remember to include a Lowes ratio test
    """
    
    # find the keypoints and descriptors with surf
    sift = cv2.SIFT_create()

    kp1,desc1 = sift.detectAndCompute(img1,None)
    kp2,desc2 = sift.detectAndCompute(img2,None)
    # draw the key on the img
    # img=cv2.drawKeypoints(img1,kp1,img1)
    
    # BFMatcher with default params
    bf = cv2.BFMatcher()
    matches = bf.knnMatch(desc1,desc2,k=2)
    # Apply ratio test
    good = []
    match_points1 = []
    match_points2 = []
    for m,n in matches:
        if m.distance < 0.7*n.distance:
            good.append([m])
            match_points1.append(kp1[m.queryIdx].pt)
            match_points2.append(kp2[m.trainIdx].pt)
#   img3 = cv2.drawMatchesKnn(gray,kp,gray2,kp2,good,None,flags=cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS)

#   plt.imshow(img3),plt.show()

    p1 = np.array(match_points1).astype(np.float32)
    p2 = np.array(match_points2).astype(np.float32)
    
    print(len(p1))
    ##### ############# ##########
    ##### Do Triangulation #######

    ##### ########################
    #project the feature points to 3D with triangulation
    
    #projection matrix for Left and Right Image
    M_left = K.dot(np.hstack((np.eye(3), np.zeros((3, 1)))))
    M_rght = K.dot(np.hstack((np.eye(3), np.array([[-baseline, 0, 0]]).T)))

    p1_flip = np.vstack((p1.T, np.ones((1, p1.shape[0]))))
    p2_flip = np.vstack((p2.T, np.ones((1, p2.shape[0]))))

    P = cv2.triangulatePoints(M_left, M_rght, p1_flip[:2], p2_flip[:2])

    # Normalize homogeneous coordinates (P->Nx4  [N,4] is the normalizer/scale)
    P = P / P[3]
    land_points = P[:3]

    return land_points.T, p1

def featureTracking(prev_img, next_img, prev_points, world_points):

    # prev_img = cv2.cvtColor(prev_img, cv2.COLOR_RGB2GRAY)
    # next_img = cv2.cvtColor(next_img, cv2.COLOR_RGB2GRAY)
    next_points, status, _ = cv2.calcOpticalFlowPyrLK(prev_img, next_img, prev_points, None, None, None, winSize=(21, 21),
                 maxLevel=3,
                 criteria=(cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 30, 0.01))
    
    # print(len(next_points))
    # print(len(world_points))
    st = status.reshape(status.shape[0])
    ##find good one
    prev_points = prev_points[st==1]
    next_points = next_points[st==1]
    world_points  = world_points[st==1]


    return world_points, prev_points, next_points

def playImageSequence(left_img, right_img, K):

    baseline = 0.54

    ##### ################################# #######
    ##### Get 3D points Using Triangulation #######
    ##### #########################################
    landmark_3D, reference_2D = extract_keypoints_surf(left_img, right_img, K, baseline)
    
    # reference
    reference_img = left_img

    # Groundtruth for plot
    truePose = getTruePose()
    traj = np.zeros((600, 600, 3), dtype=np.uint8);
    maxError = 0

    for i in range(0, 1400):
        print('image: ', i)
        curImage = getLeftImage(i)
        curImage_R = getRightImage(i)

        ##### ############################################################# #######
        ##### Calculate 2D and 3D feature correspndances in t=T-1, and t=T  #######
        ##### #####################################################################
        """
        Implement step 2.2)
        Remember this is a part of a loop, so the initial features are already
        provided in step 1)-1.3) outside the loop in 'reference_2D' and 'landmark_3D'
        """
        # the 2D landmarks at the current time = t
        landmark_3D, reference_2D, tracked_2Dpoints = featureTracking(reference_img, 
                                                                      curImage, 
                                                                      reference_2D,
                                                                      landmark_3D)
        ##### ################################# #######
        ##### Calculate relative pose using PNP #######
        ##### #########################################
        _, rvec, tvec, inliers = cv2.solvePnPRansac(landmark_3D, tracked_2Dpoints, K, distCoeffs=None)
        print(rvec, tvec)

        ##### ####################################################### #######
        ##### Get Pose and Tranformation Matrix in world coordionates #######
        ##### ###############################################################
        rot, _ = cv2.Rodrigues(rvec)
        tvec = -rot.T.dot(tvec)  # coordinate transformation, from camera to world. What is the XYZ of the camera wrt World
        inv_transform = np.hstack((rot.T, tvec))  # inverse transform. A tranform projecting points from the camera frame to the world frame

        ##### ################################# #######
        ##### Get 3D points Using Triangulation #######
        ##### #########################################
        # re-obtain the 3D points
        """
        Implement step 2.4)
        """
        landmark_3D_new, reference_2D_new = extract_keypoints_surf(curImage, curImage_R, K, baseline)
        
        #Project the points from camera to world coordinates
        reference_2D = reference_2D_new.astype('float32')
        landmark_3D = inv_transform.dot(np.vstack((landmark_3D_new.T, np.ones((1, landmark_3D_new.shape[0])))))
        landmark_3D = landmark_3D.T

        ##### ####################### #######
        ##### Done, Next image please #######
        ##### ###############################
        reference_img = curImage

        ##### ################################## #######
        ##### START OF Print and visualize stuff #######
        ##### ##########################################
        # draw images
        draw_x, draw_y = int(tvec[0]) + 300, 600-(int(tvec[2]) + 100);
        true_x, true_y = int(truePose[i][3]) + 300, 600-(int(truePose[i][11]) + 100)

        curError = np.sqrt(
            (tvec[0] - truePose[i][3]) ** 2 +
            (tvec[1] - truePose[i][7]) ** 2 +
            (tvec[2] - truePose[i][11]) ** 2)
        
        if (curError > maxError):
            maxError = curError

        print(tvec[0],tvec[1],tvec[2], rvec[0], rvec[1], rvec[2])
        print([truePose[i][3], truePose[i][7], truePose[i][11]])
        
        text = "Coordinates: x ={0:02f}m y = {1:02f}m z = {2:02f}m".format(float(tvec[0]), float(tvec[1]),
                                                                           float(tvec[2]));
        cv2.circle(traj, (draw_x, draw_y), 1, (0, 0, 255), 2);
        cv2.circle(traj, (true_x, true_y), 1, (255, 0, 0), 2);
        cv2.rectangle(traj, (10, 30), (550, 50), (0, 0, 0), cv2.FILLED);
        cv2.putText(traj, text, (10, 50), cv2.FONT_HERSHEY_PLAIN, 1, (255, 255, 255), 1, 8);

        h1, w1 = traj.shape[:2]
        h2, w2 = curImage.shape[:2]
        vis = np.zeros((max(h1, h2), w1 + w2, 3), np.uint8)
        vis[:h1, :w1, :3] = traj
        vis[:h2, w1:w1 + w2, :3] = np.dstack((np.dstack((curImage,curImage)),curImage))

        cv2.imshow("Trajectory", vis);
        k = cv2.waitKey(1) & 0xFF
        if k == 27: break


    cv2.waitKey(0)
    cv2.destroyAllWindows()
    print('Maximum Error: ', maxError)
    ##### ################################ #######
    ##### END OF Print and visualize stuff #######
    ##### ########################################

if __name__ == '__main__':
    left_img = getLeftImage(0)
    right_img = getRightImage(0)

    K = getK()

    playImageSequence(left_img, right_img, K)

942
image:  0
[[-8.34502965e-05]
 [-5.48880937e-05]
 [ 5.67322857e-05]] [[0.00075993]
 [0.00077698]
 [0.00040671]]
942
[-0.00076] [-0.00077691] [-0.00040673] [-8.34502965e-05] [-5.48880937e-05] [5.67322857e-05]
[5.551115e-17, 3.330669e-16, -4.440892e-16]
image:  1
[[-0.00233391]
 [ 0.00330903]
 [-0.00248072]] [[ 1.40322040e-04]
 [ 5.02500750e-03]
 [-6.73773044e-01]]
912
[-0.00235541] [-0.00660061] [0.67375535] [-0.00233391] [0.00330903] [-0.00248072]
[-0.04690294, -0.02839928, 0.8586941]
image:  2
[[-0.00400179]
 [ 0.00730648]
 [-0.00079479]] [[ 0.00548886]
 [ 0.00736001]
 [-1.37062042]]
940
[-0.01549487] [-0.01285308] [1.37050332] [-0.00400179] [0.00730648] [-0.00079479]
[-0.09374345, -0.05676064, 1.716275]
image:  3
[[-0.00546712]
 [ 0.01120414]
 [-0.00067545]] [[ 0.00811414]
 [ 0.0094108 ]
 [-2.08210337]]
997
[-0.03143071] [-0.02080659] [2.08179923] [-0.00546712] [0.01120414] [-0.00067545]
[-0.1406429, -0.08515762, 2.574964]
image:  4
[[-0.00620376]
 [ 0.0160319 ]
 [ 0.00084305]] [[

[ WARN:0@25.489] global /io/opencv/modules/imgcodecs/src/loadsave.cpp (239) findDecoder imread_('left/0000000101.png'): can't open/read file: check file path/integrity
[ WARN:0@25.489] global /io/opencv/modules/imgcodecs/src/loadsave.cpp (239) findDecoder imread_('right/0000000101.png'): can't open/read file: check file path/integrity


# Challenge 
The current implementation only uses features computed at the current timestep. However, as we process more images we potentially have a lot of features from previous timesteps that are still valid. The challenge is to expand the `extract_keypoints_surf(..., refPoints)` function by giving it old reference points. You should then combine your freshly computed features with the old features and remove all duplicates. This requires you to keep track of old features and 3D points.

Hint 1: look in `helpers.py` for removing duplicates.

Hint 2: you are not interested in points that are behind you, so remember to remove points that are negative in the direction you move.